In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
data = wine[['alcohol', 'sugar', 'pH']]

In [5]:
target = wine['class']

In [8]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [9]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [10]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [12]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차검증
* 3-fold 교차검증 : 훈련세트를 3 부분으로 나눠서 교차검증을 수행하는것

In [13]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.03771853, 0.05019259, 0.04819584, 0.07863188, 0.0412972 ]), 'score_time': array([0.01454377, 0.00367141, 0.01659846, 0.01016045, 0.0134387 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [14]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [15]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [19]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))
# KFold 클래스도 동일한 방식으로 사용 가능

0.8574181117533719


In [26]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [27]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [34]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [35]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [36]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [37]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [40]:
print(gs.cv_results_['params'][gs.best_index_])

{'min_impurity_decrease': 0.0001}


In [45]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
          }

In [47]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [48]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [50]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [53]:
from scipy.stats import uniform, randint

In [54]:
rgen = randint(0,10)
rgen.rvs(10)

array([4, 3, 3, 6, 3, 4, 4, 9, 9, 8])

In [55]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([102, 103,  99, 101, 101, 113,  79,  89, 116,  97]))

In [56]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.27673437, 0.13134454, 0.52550365, 0.63204404, 0.3366402 ,
       0.64155583, 0.34165922, 0.58742095, 0.75217862, 0.07359693])

In [60]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf': randint(1, 25)
          }

In [61]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params,
                        n_iter=100, n_jobs= -1, random_state= 42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b1156a29f10>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7b1156a285c0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b1156a0c3b0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b11571fba10>},
                   random_state=42)

In [63]:
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


In [64]:
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [66]:
dt = rs.best_estimator_
print(dt.score(test_input, test_target))

0.86
